# ETL9B binary -> png

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## var setting
共通で使用する変数を定義する

In [ ]:
# 各種変数設定
ETL9B_HOME = "/content/drive/MyDrive/home/school/3/ai-dev" #ルートディレクトリ
ETL9B_ZIP = ETL9B_HOME + "/zip/ETL9B.zip" # etl9bのzipファイル
UNZIP_HOME = ETL9B_HOME + "/unzip" # unzipしたファイルの場所

ETL9B_UNZIP = UNZIP_HOME + "/ETL9B"

ETL9B_PNG = ETL9B_HOME + '/png' # PNGを保存するフォルダ

## unzip
zip状態のbinaryを解凍する


In [ ]:
import shutil
import pathlib

unzip_dir = pathlib.Path(UNZIP_HOME)

# unzipフォルダが存在しないとき作成する
if not unzip_dir.exists(): 
  unzip_dir.mkdir()

shutil.unpack_archive(ETL9B_ZIP, UNZIP_HOME)

## データセット準備

### バイナリ読み込み

In [ ]:
# 出力フォルダ作成
import pathlib

png_dir = pathlib.Path(ETL9B_PNG)

if not png_dir.exists(): 
  png_dir.mkdir()

print(png_dir)

In [ ]:
# etl9bフォルダからそれぞれのパスをもってくる
import glob

files = glob.glob(ETL9B_UNZIP + "/*")

print(files)

In [ ]:
import os
import struct
from PIL import Image
from pathlib import Path

for fname in files:
  if fname == ETL9B_UNZIP + "/ETL9INFO": continue # skip info
  if fname == ETL9B_UNZIP + "/ETL9B_2": continue
  if fname == ETL9B_UNZIP + "/ETL9B_3": continue
  if fname == ETL9B_UNZIP + "/ETL9B_4": continue
  if fname == ETL9B_UNZIP + "/ETL9B_5": continue

  print('START: ' + fname)

  # ファイルを開く
  f = open(fname, 'rb')
  f.seek(0)

  ##
  cnt = 0
  out_dir = Path(ETL9B_PNG, 'ETL9B')
  while True:
    cnt += 1
    if cnt == 1: continue ## エラー起きるからスキップ。理由は調べてない
    s = f.read(576)
    if not s: break

    r = struct.unpack('>2H4s504s64x', s)
    i1 = Image.frombytes('1', (64, 63), r[3], 'raw')

    file_name = 'ETL9B_{}_{}_{}.png'.format((r[0]-1)%20+1, hex(r[1])[-4:], cnt)
    dir_name = "{}".format(hex(r[1])[-4:])
    dir = Path(out_dir, dir_name)
    # print(file_name)
    if not dir.exists():
      dir.mkdir(parents=True)
    
    png = Path(dir, file_name)

    i1.save(png, 'PNG')
